## Tiktok

Possible endpoint to use: https://developers.tiktok.com/doc/research-api-specs-query-videos?enter_method=left_navigation

Maybe not.. : https://developers.tiktok.com/products/research-api/

Maybe instead: https://pypi.org/project/TikTokApi/3.8.0/#the-search_for_hashtags-method

In [6]:
import requests
from bs4 import BeautifulSoup

def get_tiktok_videos(search_query):
    url = f"https://www.tiktok.com/search?q={search_query}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    videos = soup.find_all('div', class_='tiktok-1dnwz1b-DivItemContainer')
    for video in videos:
        print(video.get_text())

search_query = "hello"
get_tiktok_videos(search_query)


In [54]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
from TikTokApi import TikTokApi

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

async def get_tiktok_cookies_and_tokens(search_query):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        # Go to TikTok
        await page.goto('https://www.tiktok.com/')

        # Perform search
        await page.fill('input[name="q"]', search_query)
        await page.press('input[name="q"]', 'Enter')
        await page.wait_for_timeout(5000)  # Wait for the search results to load

        # Get cookies
        cookies = await context.cookies()
        await browser.close()

        # Extract necessary tokens and cookies
        ms_token = None
        verify_fp = None
        for cookie in cookies:
            if cookie['name'] == 'msToken':
                ms_token = cookie['value']
            if cookie['name'] == 's_v_web_id':
                verify_fp = cookie['value']

        return ms_token, verify_fp

def initialize_tiktok_api(ms_token, verify_fp):
    return TikTokApi(ms_token=ms_token, custom_verify_fp=verify_fp)

def get_tiktok_videos(api, search_query):
    results = api.by_search(search_query, count=10)

    for video in results:
        caption = video.get('desc', 'No caption')
        video_url = f"https://www.tiktok.com/@{video['author']['uniqueId']}/video/{video['id']}"
        print(f"Caption: {caption}")
        print(f"URL: {video_url}")

# Function to run the asyncio code
def run_async(func, *args):
    loop = asyncio.get_event_loop()
    if loop.is_running():
        return asyncio.ensure_future(func(*args))
    else:
        return loop.run_until_complete(func(*args))

# Example search query
search_query = "hello"

# Run the asynchronous function to get cookies and tokens
tokens = run_async(get_tiktok_cookies_and_tokens, search_query)
print(tokens)

# Use the tokens retrieved earlier
ms_token, verify_fp = tokens.result()
api = initialize_tiktok_api(ms_token, verify_fp)

# Fetch and print videos
get_tiktok_videos(api, search_query)


<Task pending name='Task-17' coro=<get_tiktok_cookies_and_tokens() running at C:\Users\DanielRies\AppData\Local\Temp\ipykernel_11532\3435583085.py:9>>


InvalidStateError: Result is not ready.